In [1]:
import numpy as np
import scipy.optimize as optimize
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

import finesse
from finesse.gaussian import BeamParam
from finesse.analysis.actions import RunLocks, Noxaxis, Xaxis
from finesse import ligo
from finesse.knm.maps import Map
from finesse.utilities.maps import circular_aperture, surface_point_absorber as spa

from LLOF3NoSqz import make_LLO

finesse.configure(plotting=True)

import warnings
warnings.filterwarnings('ignore')

In [2]:
def cavityPole(LLO, plot=False):
    LLO.modes(maxtem=4)
    LLOSignal = LLO.deepcopy()
    LLOSignal.parse("""
    fsig(1)

    frequency_response(
    geomspace(10, 6k, 200),  # frequency vector
    [DARM,],  # injection nodes, dof DARM
    AS.DC,  # OMC_OC.p3.o DC, AS port
    name='signal transfer function'
    )

    """)

    sol = LLOSignal.run()

    solution = np.abs(sol['signal transfer function'])
    scaled = solution.max()/solution

    ratio = 10**(3/20)
    argmax = (1/(np.abs(scaled-ratio))).argmax()
    x = np.logspace(np.log10(10), np.log10(6000), num=200)
    
#     print(f"The 3dB Signal PSD Decrement Frequency (the Pole Frequency) Is at: {} Hz")
    
    if plot:
        fig,ax = plt.subplots(1,1,figsize=(6,4))
        y = np.abs(sol["signal transfer function"])
        ax.loglog(x, y)

        # ax.set_title("Signal Transfer Function (DARM -> AS)", fontsize=13)
        # ax.axvline(x=x[argmax], ymin=0.1, ymax=scaled[argmax])
        # ax.axhline(y=scaled[argmax], xmin=x[0], xmax=x[argmax])
        ax.set_ylabel("Signal TF (DARM -> AS)", fontsize=13)
        ax.set_xlabel("fsig.f [Hz]", fontsize=13)
        ax.scatter(x[argmax], y[argmax]);
        
    return x[argmax]

In [3]:
def shotNoise(LLO, plot=False):
    LLO.modes(maxtem=4)
    LLOQNLSNoOutput = LLO.deepcopy()

    LLOQNLSNoOutput.parse("""
    ###########################################################################
    ### pendulum for QRPN
    variable Mitmy 39.420 
    variable Metmy 39.642
    variable Mitmx 39.603
    variable Metmx 39.508

    pendulum itmx_sus ITMX.mech mass=&Mitmx
    pendulum etmx_sus ETMX.mech mass=&Metmx
    pendulum itmy_sus ITMY.mech mass=&Mitmy
    pendulum etmy_sus ETMY.mech mass=&Metmy

    fsig(1)
    sgen darmx LX.h
    sgen darmy LY.h phase=180

    qnoised qnoise SRMAR.p2.o nsr=True
    qshot qshot SRMAR.p2.o nsr=True

    xaxis(darmx.f, log, 10, 6k, 100, 
                pre_step=run_locks(method='newton', display_progress=false, max_iterations=1000))

    ad AMP45 SRMAR.p2.o f=&f2
    ad AMPCar SRMAR.p2.o f=0
    """)
    outLLONoOutput = LLOQNLSNoOutput.run()

    x = np.logspace(np.log10(10), np.log10(6000), num=101)

    if plot:
        fig,ax = plt.subplots(1,1,figsize=(6,4))
        ax.loglog(x, outLLONoOutput["qnoise"].real, 'r', label='Quantum Noise')
        ax.loglog(x, outLLONoOutput["qshot"].real, 'b--', label='Shot Noise')

        ax.set_title("LLO L1", fontsize=13)
        ax.set_ylabel("ASD [W/sqrt(Hz)]", fontsize=13)
        ax.legend(loc = 'best', fontsize=13)
        ax.set_xlabel("fsig.f [Hz]", fontsize=13);
    
    return x, outLLONoOutput["qnoise"].real

In [4]:
LIGOIFO = finesse.Model()
LIGOIFO.parse_file("LIGO_degisn.txt")

LLObase = make_LLO(LIGOIFO.deepcopy())

In [5]:
#
LLO = LLObase.deepcopy()

LLO.mod1.midx = 0.135#0.133
LLO.mod2.midx = 0.1645#0.145

LLO.modes(maxtem=8)

LLO.DARM.DC = 0.00092
LLO.remove(LLO.DARM_dc_lock)

lossX = 11.25e-6 
lossY = 11.25e-6 
LLO.ETMY.set_RTL(L=4e-05+lossY, T=3.9e-06)
LLO.ETMX.set_RTL(L=4e-05+lossX, T=4e-06)

LLO.ITMX.Rc = -1958.4
LLO.ITMY.Rc = -1962.4
LLO.lensfX.value = 1/(55.8e-6)
LLO.lensfY.value = 1/(44.4e-6)

LLO.ETMX.Rc = 2259.8
LLO.ETMY.Rc = 2254.6

# q = LLO.create_mismatch(LLO.PRMAR.p1.i, w0_mm=1, z_mm=1)

delta = -0.65e-4#-1.957e-4#2.045e-4
pr2pr3 = 16.1648
LLO.lp2.L = pr2pr3*(1-delta)

# delta = 2e-4#-1.957e-4#2.045e-4
# sr2sr3 = 15.4435
# LLO.ls2.L = sr2sr3*(1-delta)

LLO.parse(f"""
bp input PRMAR.p1.i q direction=x q_as_bp=true

pd Pin PRMAR.p1.i
pd PRefl PRMAR.p1.o
pd PAS SRMAR.p2.o

ad ASC2 OMC_OC.p3.o f=0
ad ASC SRMAR.p2.o f=0
ad PAS45 SRMAR.p2.o f=&f2
ad PAS45M SRMAR.p2.o f=-&f2

ad PAS9 SRMAR.p2.o f=&f1
ad PAS9M SRMAR.p2.o f=-&f1

ad PASHG00 SRMAR.p2.o f=0 n=0 m=0

# For PRG
pd Pcirc PRM.p2.o

noxaxis(pre_step=run_locks(
            method='newton',
            display_progress=false,
            max_iterations=1000)
)

ad OMCIN OMC_IC.p1.i f=0
ad OMCREFL OMC_IC.p2.o f=0
ad OMCREFLHG00 OMC_IC.p2.o f=0 n=0 m=0

mmd modemismatchBP OMC_IC.p1.i OMC_IC.p3.o direction=x percent=false

# For Contrast Defect
pd PBS2AS BSAR2.p4.o
pd PINBS BS.p1.i

""")

kat1 = LLO.deepcopy()
out = kat1.run()

print(f"PRG: {out['Pcirc']/out['Pin']:.6f}. ##  Measurement: (42.84, 42.92)")
print(f"Prefl: {out['PRefl']:.6f} W.  ##  Measurement: (2.04, 2.05)")
print(f"PAS: {out['PAS']*1e3:.6f} mW.  ##  Measurement: (511.81, 513.82)")
print(f"PASCarrier: (After OMC) {np.abs(out['ASC2'])**2*1e3:.6f} mW.  ##  Estimate: (25, 30)")
# print(f"PASCarrier (Before OMC): {np.abs(out['ASC'])**2}")
# print(f"HG00 into OMC: {np.abs(out['PASHG00'])**2}")

print()
print(f"PAS 45 MHz: {np.abs(out['PAS45'])**2+np.abs(out['PAS45M'])**2}")
print(f"PAS 9 MHz: {np.abs(out['PAS9'])**2+np.abs(out['PAS9M'])**2}")

print(f"PAS junk light (non-HG00) in the carrier: {np.abs(out['ASC'])**2-np.abs(out['PASHG00'])**2}")

# print()
# print(f"Input beam parameter: {out['input']}")3
print()
# print(f"Full IFO mode matching (Power) is: \
# {1-np.abs(out['OMCREFL'])**2/np.abs(out['OMCIN'])**2}")
print(f"Full IFO mode matching (BP) is: \
{1-out['modemismatchBP']}")
# print(f"HOM power in the carrier at OMC REFL: {np.abs(out['OMCREFL'])**2-np.abs(out['OMCREFLHG00'])**2}")
# print(f"Carrier power going into the OMC: {np.abs(out['OMCIN'])**2}")

# print(f"Full IFO mode matching (BP) is: {1-out['modemismatchBP']}")
print()
print(f"Input beam parameter is {out['input']}")
print()
# print(f"The cavity pole is {cavityPole(LLO):.4f} Hz")
print(f"The contrast defect is {out['PBS2AS']/out['PINBS']*1e6:.2f} ppm")

PRG: 43.280200. ##  Measurement: (42.84, 42.92)
Prefl: 1.407442 W.  ##  Measurement: (2.04, 2.05)
PAS: 474.537826 mW.  ##  Measurement: (511.81, 513.82)
PASCarrier: (After OMC) 11.338380 mW.  ##  Estimate: (25, 30)

PAS 45 MHz: 0.4564226006450491
PAS 9 MHz: 0.0014986108712268138
PAS junk light (non-HG00) in the carrier: 0.004679338033863223

Full IFO mode matching (BP) is: 0.9754283197702485

Input beam parameter is BeamParam(w0=1.0645 mm, z=5.5423 m, w=2.0598 mm, Rc=7.5619 m)

The contrast defect is 846.43 ppm


In [6]:
cavityPole(LLO)

458.46535905070596